# Before Validators

More often that not we run our validators after Pydantic has had first crack at the data being deserialized - what are called **after validators**.

The main advantage of doing this, is that our validators are guaranteed that the value they receive is of the correct type and has been validated to the extent that we specified in the model.

Sometimes, however, we want to intercept the data **before** Pydantic runs its own validators - these are called "before validators".

The reason for doing this is when we need to customize the deserialization process before Pydantic does it's own validation.

For example, if we have a model with a datetime field, Pydantic's coercion of strings to datetime will work as long as the string is in a specific ISO format.

In [1]:
from datetime import datetime

from pydantic import BaseModel, field_validator, ValidationError

In [2]:
class Model(BaseModel):
    dt: datetime

In [3]:
Model(dt="2020-01-01T12:00:00")

Model(dt=datetime.datetime(2020, 1, 1, 12, 0))

In [4]:
Model(dt="2020-01-01T12:00:00Z")

Model(dt=datetime.datetime(2020, 1, 1, 12, 0, tzinfo=TzInfo(UTC)))

However, none of these will work:

In [5]:
try:
    Model(dt="2020/1/1 3:00pm")
except ValidationError as ex:
    print(ex)

1 validation error for Model
dt
  Input should be a valid datetime, invalid date separator, expected `-` [type=datetime_parsing, input_value='2020/1/1 3:00pm', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/datetime_parsing


In [6]:
try:
    Model(dt="Jan 1, 2020 3:00pm")
except ValidationError as ex:
    print(ex)

1 validation error for Model
dt
  Input should be a valid datetime, invalid character in year [type=datetime_parsing, input_value='Jan 1, 2020 3:00pm', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/datetime_parsing


We can write our own custom validator to basically deserialize these strings, but we cannot use an **after** validator, since Pydantic's serializers will raise a validation error before our custom validator is executed.

And so, we can use a **before** validator.

Let's first write some Python code that will be able to parse these various date strings into proper datetime objects.

We'll use the `python-dateutil` 3rd party library to do this.

You'll need to install in in your virtual environment, and docs for that library are located [here](https://github.com/dateutil/dateutil)

In particular, we'll use that library's parser.

In [7]:
from dateutil.parser import parse

In [8]:
parse("2020/1/1 3pm")

datetime.datetime(2020, 1, 1, 15, 0)

In [9]:
parse("Jan 1, 2020 3pm")

datetime.datetime(2020, 1, 1, 15, 0)

However, the function will only work if passed a string, not a datetime object:

In [10]:
try:
    parse(datetime(2020, 1, 1, 15, 0, 0))
except TypeError as ex:
    print(ex)

Parser must be a string or character stream, not datetime


So, when we use our before validator, we'll receive the raw value, which could be a string, a datetime object, or any other type of object that is passed in.  

We'll have to account for that in our validator.

Defining a **before** validator, is done the same was as an **after** validator, we just have to add an argument to indicate the validator is a **before** validator *the default is an **after** validator).

Technically we don't have to return the final (model) type from our validator, since the Pydantic validators will run after, but we could if we wanted to. In this case, we're going to attempt to parse the value if it is a string, otherwise we'll just forward the value, whatever type it is, and let Pydantic handle non-string input values.

In [11]:
from typing import Any


class Model(BaseModel):
    dt: datetime

    @field_validator("dt", mode="before")
    @classmethod
    def parse_datetime(cls, value: Any):
        if isinstance(value, str):
            print("parsing string")
            try:
                return parse(value)
            except Exception as ex:
                raise ValueError(str(ex))
        print("pass through...")
        return value

In [12]:
Model(dt="2020/1/1 3pm")

parsing string


Model(dt=datetime.datetime(2020, 1, 1, 15, 0))

Any other data type will be passed through our custom validator as-is, and then handled by Pydantic's validators:

In [13]:
Model(dt=datetime(2020, 1, 1))

pass through...


Model(dt=datetime.datetime(2020, 1, 1, 0, 0))

In [14]:
try:
    Model(dt=[1, 2, 3])
except ValidationError as ex:
    print(ex)

pass through...
1 validation error for Model
dt
  Input should be a valid datetime [type=datetime_type, input_value=[1, 2, 3], input_type=list]
    For further information visit https://errors.pydantic.dev/2.5/v/datetime_type


So, just keep in mind, with **before** validators, our validator receives the **raw** value as it was received from the data being deserialized, or from a previous before validator if there was one.

We can perform any type of validation, or transformation we want, and return a value. Once it's returned, the next before validator is excecuted, and if none are present, Pydantic will then take over the rest of the validation, followed by after validators (if any).

One last thing to look at is the order of execution when we have multiple before validators.

Remember from the lecture video, we said that Pydantic will execute before validators from bottom to top of the definition order.

Let's take a look at that quickly:

In [15]:
class Model(BaseModel):
    number: int

    @field_validator("number", mode="before")
    @classmethod
    def validator_1(cls, value):
        print("running validator_1")
        return value

    @field_validator("number", mode="before")
    @classmethod
    def validator_2(cls, value):
        print("running validator_2")
        return value

    @field_validator("number", mode="before")
    @classmethod
    def validator_3(cls, value):
        print("running validator_3")
        return value

    

In [16]:
Model(number=1)

running validator_3
running validator_2
running validator_1


Model(number=1)

As you can see, the order is bottom to top.